In [1]:
#Writing Replacement Function
import pandas as pd
import numpy as np
from utils.Dataframe_tools import PSD 
from utils import iterateSubDirFiles

#Setting the Values
IO='input files\KPBom.xlsx'

find_remove=[{
    '91840054':'99717392',
    '91840056':'99717485',
    '98441969':'92758523',
    '96698939':'92758528',     # in KPbom.xlsx 
    '96698968':'92758121',     # in KPbom.xlsx
    '96768950':'92758767',     # in KPbom.xlsx
    '99413027':'92758120',
    '1020-3/4':'1020-5/6',
    '1020-3_4':'1020-5_6',
    'Desc-1020-3_4-KP_4':'Desc-1020-5_6-KP_6',
    'Desc-1020-3_4-KP_3':'Desc-1020-5_6-KP_5'
},]

def find_and_replace(df:pd.DataFrame,to_replace,new_value)->None:
    df.replace(to_replace=to_replace,value=new_value,regex=True,inplace=True)

def change_psd_file(filename,function,*args,**kwargs)->list:
    sheet_list=pd.ExcelFile(filename).sheet_names
    df_dict={}
    for sheet in sheet_list:
        try:
            df=PSD(io=filename,sheet_name=sheet)
            function(df.psd_data,*args,**kwargs)
            df_dict.update({sheet:df})
        except TypeError:
            print(sheet.title()+" is not in PSD format.")
    return df_dict
        

dfs=change_psd_file(filename=IO,function=find_and_replace,to_replace=list(find_remove[0].keys()),new_value=list(find_remove[0].values()))



Info is not in PSD format.
Bom Breakdown is not in PSD format.
Misc is not in PSD format.
Seals is not in PSD format.
Couplings is not in PSD format.


In [2]:
case=dfs['Case']
data=case.psd_data
header=case.header_data

In [12]:
from utils.Dataframe_tools import PSD
filename='input files\Copy of Price-master.xlsx'
sheet='A'
df=PSD(io=filename,sheet_name=sheet)
data=df.psd_data
header=df.header_data

In [13]:
end_row = data[data.iloc[:,0] == '[END]'].index.to_list()[0]

In [2]:


endr=header[header.iloc[:,0].str.startswith("[")].index[0]
classes=header[:endr].values.tolist()
types=header.iloc[endr].tolist()
arr=[]
#Now I need to iterate over the classes
for row in classes:
    dct={}
    dct2={}
    new_data=data
    new_data.columns=row
    column_numbers = [x for x in range(new_data.shape[1])]
    for j,col in enumerate(row):
        if j==0:
            dct.update({'class':col})
        if str(col).strip().lower()=='nan':
            column_numbers .remove(j)
        else:
            if j==0:
                column_numbers.remove(0)
            else:
                dct2.update({col:types[j]})
        if str(col).strip().lower()=='id':
            data2=data.groupby(col)
            IDs=list(data2.groups.keys())
            dct.update({'instances':IDs})
    new_data=new_data.iloc[:, column_numbers]
    dct.update({'data':new_data})
    dct.update({'DataTypes':dct2})
    arr.append(dct)

    


In [5]:
import lxml.etree as ET
from typing import ParamSpec,Callable,Concatenate,TypeVar


def create_control_node(class_name:str,column_name:str,pointer_merge_dict:dict={'merge':'true','position':"end",'offset':"0"})->ET.Element:
    """Function Description"""
    #May need to make sure that the pointer merge dict contains valid entries
    ControlNodeElement=ET.Element('controlnode') 
    classcontroldata=ET.SubElement(ControlNodeElement,'classcontroldata',attrib={'class':class_name})
    attributecontroldata=ET.SubElement(classcontroldata,'attributecontroldata',{'attribute':column_name})
    ET.SubElement(attributecontroldata,'pointermergedata',pointer_merge_dict)
    return ControlNodeElement



In [13]:
import lxml.etree as ET

"""
Instances
    |_Class
        |_Instance
            |_Class
                |_Ref
            |_Chartype
            |_attribute (data)

"""

"""Imports: PSD data with header removed """

# num=1

# #Outputs from my Code
# psd_data=arr[num]['data'].iloc[:-1,:]
# class_ref_name=arr[num]['class']
# DataTypes=arr[num]['DataTypes']
# Instances={x:ET.Element('instance',{'name':x}) for x in arr[num]['instances']} #List of elements that can appended to

#setting up control node array 
cn_arr=[]
instance_ele=ET.Element('instances')


for row in arr:
    #Constants for the row
    class_ref_name=row['class']
    psd_data=row['data'].iloc[:-1,:]
    DataTypes=row['DataTypes']
    Instances={x:ET.Element('instance',{'name':x}) for x in row['instances']}

    # Iterate through dataframe in correct order to produce XML instance
    root = ET.Element('class',{'name':class_ref_name})  # Root element

    # Sets Class for each row/entry from PSD (Should be the same for all entries)
    class_attr = {"name": class_ref_name}

    # Sets chartype, which is always a doublebyte
    chartype_attr = {"value": "doublebyte"}

    for index, col in psd_data.iterrows():
        #Need to use the instance assoicated with the row being looked at
        instance=Instances[col['ID']]

        #We need to check to see if the instance already exists in root
        find_txt=f"instance[@name='{instance.attrib['name']}']"
        if root.find(find_txt)==None:
            root.append(instance)
            # Inserts Class for each instance in PSD
            class_entry = ET.SubElement(instance, "class")
            class_sub_entry = ET.SubElement(class_entry, "ref", attrib=class_attr)
            # Inserts chartype attribute
            chartype = ET.SubElement(instance, "chartype", chartype_attr)

        # # Loop through header dictionary for attribute types
        for k, v in  DataTypes.items():
            find_txt=f"attribute[@name='{k}']"
            # Need to insert if statement where pointers receive sub-child
            if k=='ID':
                if 'pointer' in v:
                    v='text'
            else:
                if v=='pointer-merge':
                    v='pointer'
                    try:
                        next(item for item in cn_arr if item["class"] == class_ref_name and item['attribute'] == k)
                    except StopIteration:
                        cn_arr.append({'element':create_control_node(class_ref_name,k),'class':class_ref_name,'attribute':k})
            if v == "text":
                attr_dict = {"name": k, "type": v, "value": psd_data.at[index, k].__str__()}
                if instance.find(find_txt)==None:
                    attr_entry = ET.SubElement(instance, "attribute", attr_dict) #This makes a new element each time and I want it to only make a new element
                else:
                    attr_entry=instance.find(find_txt) 
            elif "pointer" in str(v): # Whats the difference between pointer and pointer-merge? May have to change this. 
                attr_dict = {"name": k, "type": v}
                if instance.find(find_txt)==None:
                    attr_entry = ET.SubElement(instance, "attribute", attr_dict)
                else:
                    attr_entry=instance.find(find_txt)
                pointer_dict = {"value": psd_data.at[index, k].__str__()}
                if instance.find(f"./attribute/ref[@value='{pointer_dict['value']}']")==None:
                    pointer_sub_entry = ET.SubElement(attr_entry, "ref", attrib=pointer_dict)
            elif v == "double":
                attr_dict = {"name": k, "type": v, "value": psd_data.at[index, k].__str__()}
                if instance.find(find_txt)==None:
                    attr_entry = ET.SubElement(instance, "attribute", attr_dict) #This makes a new element each time and I want it to only make a new element
                else:
                    attr_entry=instance.find(find_txt)
    
    instance_ele.append(root)


root=ET.Element('FirepondSPCDatabase',attrib={'version':'3.0'}) #Creates the Root Node

for node in cn_arr:
    root.append(node['element'])

root.append(instance_ele)

with open('test.xml','wb') as doc:
    doc.write(ET.tostring(root,encoding='UTF-8',xml_declaration=True,standalone=True,pretty_print=True))

In [66]:
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'UTF-8')
    reparsed = minidom.parseString(rough_string)
    print(reparsed.toprettyxml(indent="  "))


In [16]:
from pandas import DataFrame,Series
from typing import Any

def remove_from_df(df:DataFrame,cols_to_remove_from:list[str],items_to_remove:list[Any])->tuple[DataFrame, DataFrame]:
    #Creating String List
    str_list = [str(numstr) for numstr in items_to_remove]
    
    #Converts BOM column into the same datatype as the removal list
    psd_data=df.astype({items_to_remove:str_list.__getitem__(0).__class__}) 
    
    #Creating Groups for Grouping
    groups = psd_data.groupby(cols_to_remove_from) 
    
    df_list_to_remove = [] # will hold list of dataframes to be removed. Will concatenate to 1 dataframe later
    df_list_to_keep   = [] # will hold list of dataframes to remain in PSD. Will concatenate to 1 dataframe later

    # Iterates through each sub-group, checking if there is a pn that meets criteria for removal
    for _, frame in groups:
        if any(frame[items_to_remove].isin(str_list)):   # Finding matching partnumbers to remove
            # Need to add this sub-group to a "removed dataframe"
            df_list_to_remove.append(frame)
        else:
            # Need to retain this sub-group, add to a "keep dataframe"
            df_list_to_keep.append(frame)
    # Concatenating list of dfs to single dfs.
    if len(df_list_to_remove) > 0:
        removals = pd.concat(df_list_to_remove)
    else:
        removals = pd.DataFrame(np.nan, index=[0], columns=psd_data.columns)
    keep = pd.concat(df_list_to_keep)
    
    return removals, keep

def check_df(df:DataFrame,cols_to_search:list[str]=["ID"],items_to_find:list=None)->tuple[bool,list]:
    matches=remove_from_df(df,cols_to_search,items_to_find)
    if matches[0].size>1:
        return True,matches[0]['ID'].to_list()
    elif matches[0]==1:
        return False,[]
    else:
        raise Exception(f"Can't have a dataframe with negative rows.")